In [55]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

import numpy as np


from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV





In [2]:
data = pd.read_csv('final_wec_data.csv')

/tmp/ipykernel_1390/3317488320.py:1: DtypeWarning: Columns (26,31,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('final_wec_data.csv')


In [3]:
# data = data.drop('Unnamed: 0', axis=1)
# #  Dropping the extra index col

# data = data[data['year'] >= 2017].reset_index(drop=True)
# # # Dropping data from prior to 2017 as driver_ratings data was not available

# data = data.drop(['lap_number', 'car_number', 'lap_number', 'driver_number', 'lap_time', 'elapsed', 'hour', 's1_large', 's2_large', 's3_large', 'driver_name', 'pit_time', 'group', 'team', 'manufacturer', 'season', 'vehicle', 'team_no','lap_time_ms', 'engine', 'driver_stint',
#            'team_stint', 'team_stint_no', 'interval_ms', 'interval', 'elapsed_ms', 'position', 'gap', 'elapsed_s', ], axis=1)
# # Initial columns dropped because not needed/not useable for modeling. Remaining columns will be further assessed 

In [4]:
data.columns

Index(['Unnamed: 0', 'car_number', 'driver_number', 'lap_number', 'lap_time',
       'lap_improvement', 'crossing_finish_line_in_pit', 's1',
       's1_improvement', 's2', 's2_improvement', 's3', 's3_improvement',
       'lap_avg_kph', 'elapsed', 'hour', 's1_large', 's2_large', 's3_large',
       'lap_max_kph', 'driver_name', 'pit_time', 'class', 'group', 'team',
       'manufacturer', 'season', 'year', 'circuit', 'round', 'vehicle',
       'flag_at_fl', 'team_no', 'lap_time_ms', 'lap_time_s', 'engine',
       'driver_stint_no', 'driver_stint', 'team_stint_no', 'team_stint',
       'elapsed_ms', 'position', 'class_position', 'interval_ms', 'interval',
       'gap', 'class_interval', 'class_gap', 'elapsed_s', 'rating',
       'rolling_5l_avg'],
      dtype='object')

In [5]:
def df_for_ml(df):

    df = df.drop('Unnamed: 0', axis=1)
    df = data[data['year'] >= 2017].reset_index(drop=True)
    # Dropping data from prior to 2017 as driver_ratings data was not available
    df = df.drop(['lap_number', 'car_number', 'lap_number', 'driver_number', 'lap_time', 'elapsed', 'hour', 's1_large', 's2_large', 's3_large', 'driver_name', 'pit_time', 'group', 'team', 'manufacturer', 'season', 'vehicle', 'team_no','lap_time_ms', 'engine', 'driver_stint',
           'team_stint', 'team_stint_no', 'interval_ms', 'interval', 'elapsed_ms', 'position', 'gap', 'elapsed_s'], axis=1)
    # Initial columns dropped because not needed/not useable for modeling. Remaining columns will be further assessed 
    
    return df




In [6]:
data = df_for_ml(data)

In [7]:
def fixing_kept_cols(df):
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].fillna(0)
    df['crossing_finish_line_in_pit'] = df['crossing_finish_line_in_pit'].replace('B', 1)
#   B represents a car crossing the start/finish line in pitlane, NaN means the car was on track. Values are replaced with binary 0 for on-track, 1 for in-pit

    df = df[df['class']!='INNOVATIVE CAR']
#     removing the innovative class to eliminate outlier data
    
    
    df['circuit'] = df['circuit'].replace({'SPA FRANCORCHAMPS':'Spa', 'BAHRAIN INTERNATIONAL CIRCUIT':'Bahrain', 'FUJI SPEEDWAY':'Fuji', 'SHANGHAI INTERNATIONAL CIRCUIT':'Shanghai', 'CIRCUIT OF THE AMERICAS':'COTA',
    'AUTODROMO HERMANOS RODRIGUEZ':'Mexico', 'LE MANS 2018':'Le Mans', 'SPA FRANCORCHAMPS 2019':'Spa','LE MANS 2019':'Le Mans', 'BAHRAIN INTERNATIONAL CIRCUIT 2019': 'Bahrain',
    'BAHRAIN INTERNATIONAL CIRCUIT 2020':'Bahrain', 'AUTODROMO DO ALGARVE':'Portimao', 'AUTODROMO NAZIONALE DI MONZA':'Monza', 'BAHRAIN INTERNATIONAL CIRCUIT 6 HOURS':'Bahrain', 'BAHRAIN INTERNATIONAL CIRCUIT 8 HOURS':'Bahrain'})
    df['circuit'] = df['circuit'].str.lower() 
#     Replacing the long-form and/or year appended circuit names with the common-use short versions. Also changes all to lower-case


    df = df.drop('flag_at_fl', axis=1)
#     df in column only exists for season 2022, some value meanings can not be determined.

    return df


In [8]:
data = fixing_kept_cols(data)

/tmp/ipykernel_1390/936137902.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['circuit'] = df['circuit'].replace({'SPA FRANCORCHAMPS':'Spa', 'BAHRAIN INTERNATIONAL CIRCUIT':'Bahrain', 'FUJI SPEEDWAY':'Fuji', 'SHANGHAI INTERNATIONAL CIRCUIT':'Shanghai', 'CIRCUIT OF THE AMERICAS':'COTA',
/tmp/ipykernel_1390/936137902.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['circuit'] = df['circuit'].str.lower()


In [9]:
data.isna().sum()/len(data)*100

#small percentages of nan values can be safely dropped without significant effect. 
#12.68% of laps were run by a driver for which we have no rating info

Unnamed: 0                      0.000000
lap_improvement                 0.000000
crossing_finish_line_in_pit     0.000000
s1                              0.007801
s1_improvement                  0.000000
s2                              0.047584
s2_improvement                  0.000000
s3                              0.039783
s3_improvement                  0.000000
lap_avg_kph                     0.000000
lap_max_kph                     0.249620
class                           0.000000
year                            0.000000
circuit                         0.000000
round                           0.000000
lap_time_s                      0.000000
driver_stint_no                 0.000000
class_position                  0.000000
class_interval                  0.000000
class_gap                       0.000000
rating                         12.686922
rolling_5l_avg                  0.000000
dtype: float64

In [10]:
data = data.dropna().reset_index(drop=True)
# Dropping NaN values to remove rows with missing data

#This will be included into the fixing_kept_cols function when the code is refactored

In [11]:
data.isna().sum()/len(data)*100

Unnamed: 0                     0.0
lap_improvement                0.0
crossing_finish_line_in_pit    0.0
s1                             0.0
s1_improvement                 0.0
s2                             0.0
s2_improvement                 0.0
s3                             0.0
s3_improvement                 0.0
lap_avg_kph                    0.0
lap_max_kph                    0.0
class                          0.0
year                           0.0
circuit                        0.0
round                          0.0
lap_time_s                     0.0
driver_stint_no                0.0
class_position                 0.0
class_interval                 0.0
class_gap                      0.0
rating                         0.0
rolling_5l_avg                 0.0
dtype: float64

In [12]:
data.head()

,Unnamed: 0,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,s3,s3_improvement,lap_avg_kph,lap_max_kph,class,year,circuit,round,lap_time_s,driver_stint_no,class_position,class_interval,class_gap,rating,rolling_5l_avg
0,246956,0,0,41.322,0,51.911,0,33.424,0,167.7,242.7,LMGTE Am,2017,silverstone,1,126.657,1,1.0,0.000,0.000,S,126.6570
1,246957,0,0,41.504,0,52.321,0,33.244,0,167.2,249.4,LMGTE Am,2017,silverstone,1,127.069,1,2.0,0.412,0.412,P,124.8002
2,246958,0,0,42.036,0,52.346,0,33.114,0,166.6,246.0,LMGTE Am,2017,silverstone,1,127.496,1,3.0,0.427,0.839,S,135.6296
3,246959,0,0,42.815,0,52.290,0,32.766,0,166.1,248.3,LMGTE Am,2017,silverstone,1,127.871,1,4.0,0.375,1.214,G,125.1642
4,246960,0,0,42.827,0,54.325,0,34.207,0,161.7,242.7,LMGTE Am,2017,silverstone,1,131.359,1,5.0,3.488,4.702,B,124.4692


In [13]:
data.dtypes

Unnamed: 0                       int64
lap_improvement                  int64
crossing_finish_line_in_pit      int64
s1                             float64
s1_improvement                   int64
s2                             float64
s2_improvement                   int64
s3                             float64
s3_improvement                   int64
lap_avg_kph                    float64
lap_max_kph                    float64
class                           object
year                             int64
circuit                         object
round                            int64
lap_time_s                     float64
driver_stint_no                  int64
class_position                 float64
class_interval                 float64
class_gap                      float64
rating                          object
rolling_5l_avg                 float64
dtype: object

In [14]:
data.describe()

,Unnamed: 0,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,s3,s3_improvement,lap_avg_kph,lap_max_kph,year,round,lap_time_s,driver_stint_no,class_position,class_interval,class_gap,rolling_5l_avg
count,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000,223221.000000
mean,372076.886745,0.009582,0.060966,44.044375,0.018184,66.645666,0.018229,70.840311,0.018179,183.043856,272.804617,2019.232698,4.302821,181.687327,3.604800,6.375135,118.440143,406.533891,184.802741
std,75091.609542,0.139702,0.239269,87.648910,0.166460,30.360915,0.166966,54.513385,0.166823,33.154265,49.356669,1.718298,2.324646,125.793155,3.021975,4.947636,666.909006,1008.045708,109.093043
min,246956.000000,0.000000,0.000000,18.809000,0.000000,25.517000,0.000000,18.052000,0.000000,3.100000,19.400000,2017.000000,1.000000,85.603000,0.000000,1.000000,0.000000,0.000000,86.116200
25%,306503.000000,0.000000,0.000000,33.874000,0.000000,42.940000,0.000000,36.366000,0.000000,163.200000,255.900000,2018.000000,2.000000,117.600000,1.000000,3.000000,2.372000,15.698000,119.731200
50%,368214.000000,0.000000,0.000000,36.560000,0.000000,63.015000,0.000000,48.153000,0.000000,180.900000,282.900000,2019.000000,4.000000,145.266000,2.000000,5.000000,16.912000,90.497000,153.333800
75%,433960.000000,0.000000,0.000000,38.531000,0.000000,85.493000,0.000000,105.573000,0.000000,208.300000,298.800000,2021.000000,6.000000,232.876000,5.000000,9.000000,71.033000,364.172000,236.383600
max,503679.000000,3.000000,1.000000,7255.675000,3.000000,3646.889000,3.000000,3157.259000,3.000000,248.600000,350.100000,2022.000000,9.000000,9906.273000,19.000000,27.000000,22046.060000,24705.360000,9906.273000


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223221 entries, 0 to 223220
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   223221 non-null  int64  
 1   lap_improvement              223221 non-null  int64  
 2   crossing_finish_line_in_pit  223221 non-null  int64  
 3   s1                           223221 non-null  float64
 4   s1_improvement               223221 non-null  int64  
 5   s2                           223221 non-null  float64
 6   s2_improvement               223221 non-null  int64  
 7   s3                           223221 non-null  float64
 8   s3_improvement               223221 non-null  int64  
 9   lap_avg_kph                  223221 non-null  float64
 10  lap_max_kph                  223221 non-null  float64
 11  class                        223221 non-null  object 
 12  year                         223221 non-null  int64  
 13 

In [16]:
data['rating'].value_counts()

P    101398
G     47668
S     43882
B     30273
Name: rating, dtype: int64

In [17]:
data['circuit'].unique()

array(['silverstone', 'spa', 'le mans', 'nurburgring', 'mexico', 'cota',
       'fuji', 'shanghai', 'bahrain', 'sebring', 'portimao', 'monza'],
      dtype=object)

In [18]:
data['class'].unique()

array(['LMGTE Am', 'LMGTE Pro', 'LMP1', 'LMP2', 'HYPERCAR'], dtype=object)

In [19]:
# obs = data.select_dtypes(object).drop('rating', axis=1)

# class_one_hot = pd.get_dummies(obs['class'], prefix='class')

# circuit_one_hot = pd.get_dummies(obs['circuit'], prefix='circuit')

# ob_feats = pd.concat([class_one_hot, circuit_one_hot], axis=1)

# ob_feats

In [20]:
# cont_nums = data.select_dtypes('number')
# cont_nums = cont_nums.drop(['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position'], axis=1)

In [21]:
# disc_nums = data[['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position']]
# disc_nums

In [22]:
def separate_df(df):
    cont_nums = df.select_dtypes('number')
    cont_nums = cont_nums.drop(['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position'], axis=1)
    
    disc_nums = df[['lap_improvement', 'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement', 's3_improvement', 'year', 'round', 'driver_stint_no', 'class_position']]
    
    obs = data.select_dtypes(object).drop('rating', axis=1)

    
    
    
    target = df['rating']
    
    return cont_nums, disc_nums, obs, target

In [23]:
def encode_obs(obs):
    class_one_hot = pd.get_dummies(obs['class'], prefix='class')
    circuit_one_hot = pd.get_dummies(obs['circuit'], prefix='circuit')
    
    ob_feats = pd.concat([class_one_hot, circuit_one_hot], axis=1)
    
    return ob_feats


In [24]:
cont_nums, disc_nums, obs, target = separate_df(data)

In [25]:
obs

,class,circuit
0,LMGTE Am,silverstone
1,LMGTE Am,silverstone
2,LMGTE Am,silverstone
3,LMGTE Am,silverstone
4,LMGTE Am,silverstone
...,...,...
223216,HYPERCAR,le mans
223217,HYPERCAR,le mans
223218,HYPERCAR,le mans
223219,HYPERCAR,le mans


In [26]:
ob_feats = encode_obs(obs)

In [27]:
ob_feats.head()

,class_HYPERCAR,class_LMGTE Am,class_LMGTE Pro,class_LMP1,class_LMP2,circuit_bahrain,circuit_cota,circuit_fuji,circuit_le mans,circuit_mexico,circuit_monza,circuit_nurburgring,circuit_portimao,circuit_sebring,circuit_shanghai,circuit_silverstone,circuit_spa
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [28]:
# s_scaler = StandardScaler()
# num_s_scal = s_scaler.fit_transform(nums)
# num_s_scal = pd.DataFrame(num_s_scal)
# num_s_scal

In [29]:
# mm_scaler = MinMaxScaler()
# num_mm_scal = mm_scaler.fit_transform(nums)
# num_mm_scal = pd.DataFrame(num_mm_scal)
# num_mm_scal

In [30]:
# abs_scaler = MaxAbsScaler()
# num_abs_scal = abs_scaler.fit_transform(nums)
# num_abs_scal = pd.DataFrame(num_abs_scal)
# num_abs_scal

In [31]:
def make_scaled_sets(df):
    from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
    
    s_scaler = StandardScaler()
    num_s_scal = s_scaler.fit_transform(df)
    num_s_scal = pd.DataFrame(num_s_scal)
    
    mm_scaler = MinMaxScaler()
    num_mm_scal = mm_scaler.fit_transform(df)
    num_mm_scal = pd.DataFrame(num_mm_scal)
    
    abs_scaler = MaxAbsScaler()
    num_abs_scal = abs_scaler.fit_transform(df)
    num_abs_scal = pd.DataFrame(num_abs_scal)
    
    
    return num_s_scal, num_mm_scal, num_abs_scal

In [32]:
num_s_scal, num_mm_scal, num_abs_scal = make_scaled_sets(cont_nums)

In [33]:
# num_s_scal.head()

In [34]:
# num_mm_scal.head()

In [35]:
# num_abs_scal.head()

In [36]:
features = pd.concat([num_s_scal, disc_nums, ob_feats], axis=1)



In [37]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 55)


In [45]:
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier(max_depth=2, random_state=0)

In [46]:
y_pred = rf.predict(X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [49]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.48788660717485577


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# create a Random Forest Classifier object
rfc = RandomForestClassifier()

# create a GridSearchCV object
grid_search = GridSearchCV(rfc, param_grid=param_grid, cv=5)

# fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklea